In [ ]:
import torchvision
import torch

In [ ]:
import torchvision.models as models

In [ ]:
vgg2 = models.vgg19(pretrained=True)

In [ ]:
i = torch.rand(1, 3, 5, 5)
output = vgg2.features[:1](i)
print(output.shape)

# Online - Test

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import models, transforms
from PIL import Image
import numpy as np
import os, glob
import random
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
class SetSeed:
    def setter(self):
        torch.manual_seed(42)
        random.seed(42)
        np.random.seed(42)

In [3]:
SetSeed().setter()

In [4]:
# data_dir = r"C:\MyDrive\work\self_project\image_recomendation\api-test-images"  # train
# features_dir = r"C:\MyDrive\work\self_project\image_recomendation\api-test-images\features"

In [5]:
data_dir = "../../api-test-images/"
features_dir = r"C:\MyDrive\work\self_project\image_recomendation\api-test-images\features"

In [6]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        VGG = models.vgg19(pretrained=True)
        self.feature = VGG.features
        self.classifier = nn.Sequential(*list(VGG.classifier.children())[:-3])
        pretrained_dict = VGG.state_dict()
        model_dict = self.classifier.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict)
        self.classifier.load_state_dict(model_dict)
 
    def forward(self, x):
        output = self.feature(x)
        output = output.view(output.size(0), -1)
        output = self.classifier(output)
        return output

In [7]:
model = Encoder()

In [8]:
def extractor(img_path, net, use_gpu):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()]
    )
 
    img = Image.open(img_path)
    img = transform(img)
    print(img.shape)
 
    x = Variable(torch.unsqueeze(img, dim=0).float(), requires_grad=False)
    print(x.shape)
 
    if use_gpu:
        x = x.cuda()
        net = net.cuda()
    y = net(x).cpu()
    y = torch.squeeze(y)
    y = y.data.numpy()
    print(y.shape)
#     np.savetxt(saved_path, y, delimiter=',')
    return y


In [9]:
extensions = ['jpg', 'jpeg', 'JPG', 'JPEG']

In [10]:
    files_list = []
    x = os.walk(data_dir)
    for path, d, filelist in x:
        for filename in filelist:
            file_glob = os.path.join(path, filename)
            files_list.extend(glob.glob(file_glob))
 
    print(files_list)

['../../api-test-images/7688415.411.jpg', '../../api-test-images/7688415.93554.jpg', '../../api-test-images/8016025.7294.jpg', '../../api-test-images/8021256.585.jpg', '../../api-test-images/8021257.158.jpg', '../../api-test-images/8084918.154841.jpg', '../../api-test-images/test.jpg']


In [11]:
filename = []
features = []
for x_path in files_list:
        print("x_path" + x_path)
        file_name = x_path.split('/')[-1]
        bottleneck_features = extractor(x_path, model, False)
        filename.append(x_path)
        features.append(list(bottleneck_features.reshape(1,-1)))

x_path../../api-test-images/7688415.411.jpg
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])


C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


(4096,)
x_path../../api-test-images/7688415.93554.jpg
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
(4096,)
x_path../../api-test-images/8016025.7294.jpg
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
(4096,)
x_path../../api-test-images/8021256.585.jpg
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
(4096,)
x_path../../api-test-images/8021257.158.jpg
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
(4096,)
x_path../../api-test-images/8084918.154841.jpg
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
(4096,)
x_path../../api-test-images/test.jpg
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
(4096,)


In [12]:
# features[0]

In [13]:
for f in range(len(features)):
    print(filename[f])
    print(cosine_similarity(features[3], features[f]))
    print("**--**"*20)

../../api-test-images/7688415.411.jpg
[[0.6716096]]
**--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--**
../../api-test-images/7688415.93554.jpg
[[0.5857068]]
**--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--**
../../api-test-images/8016025.7294.jpg
[[0.42096156]]
**--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--**
../../api-test-images/8021256.585.jpg
[[0.99999994]]
**--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--**
../../api-test-images/8021257.158.jpg
[[0.68518484]]
**--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--****--**
../../api-test-images/8084918.154841.jpg
[[0.49411142]]
**--****--****--****--****--****--****--****--****--****--****--****--***

In [26]:
data = pd.DataFrame({"FileName": filename, "features": features})

In [27]:
# features[0]

In [28]:
data.head()

,FileName,features
0,../../api-test-images/7688415.411.jpg,"[[-1.9931037, -1.8752654, -2.9238546, 0.274329..."
1,../../api-test-images/7688415.93554.jpg,"[[-1.5990303, -2.5206456, -1.9668546, 0.257537..."
2,../../api-test-images/8016025.7294.jpg,"[[0.24607146, -2.8339028, -1.4787565, 0.940413..."
3,../../api-test-images/8021256.585.jpg,"[[-1.8691487, -0.112221465, -1.5841146, 0.0016..."
4,../../api-test-images/8021257.158.jpg,"[[-0.8288594, -0.22768264, -1.1380577, -1.1019..."


In [29]:
data['features'].tolist()[0][0]

array([-1.9931037 , -1.8752654 , -2.9238546 , ...,  0.26278403,
       -2.3719916 , -0.4103165 ], dtype=float32)

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   FileName  7 non-null      object
 1   features  7 non-null      object
dtypes: object(2)
memory usage: 240.0+ bytes


In [45]:
data.to_pickle('d.pkl')
data.to_csv('d.csv', index=False)

In [80]:
data_list_dict = data.to_dict('records')

In [83]:
data_list_dict

[{'FileName': '../../api-test-images/7688415.411.jpg',
  'features': [array([-1.9931037 , -1.8752654 , -2.9238546 , ...,  0.26278403,
          -2.3719916 , -0.4103165 ], dtype=float32)]},
 {'FileName': '../../api-test-images/7688415.93554.jpg',
  'features': [array([-1.5990303, -2.5206456, -1.9668546, ...,  0.6526101, -1.3538702,
          -1.397386 ], dtype=float32)]},
 {'FileName': '../../api-test-images/8016025.7294.jpg',
  'features': [array([ 0.24607146, -2.8339028 , -1.4787565 , ...,  0.11486907,
          -0.39319983, -1.0733975 ], dtype=float32)]},
 {'FileName': '../../api-test-images/8021256.585.jpg',
  'features': [array([-1.8691487 , -0.11222146, -1.5841146 , ...,  1.1097003 ,
          -1.1005354 , -0.22999443], dtype=float32)]},
 {'FileName': '../../api-test-images/8021257.158.jpg',
  'features': [array([-0.8288594 , -0.22768264, -1.1380577 , ...,  0.97883475,
          -1.4303892 , -0.63086367], dtype=float32)]},
 {'FileName': '../../api-test-images/8084918.154841.jpg',


In [84]:
gg = [{'name': 'a', "similarity":0.2}, {'name': 'a', "similarity":0.99}, {'name': 'a', "similarity":0.24}, {'name': 'a', "similarity":0.52}]

In [85]:
gg

[{'name': 'a', 'similarity': 0.2},
 {'name': 'a', 'similarity': 0.99},
 {'name': 'a', 'similarity': 0.24},
 {'name': 'a', 'similarity': 0.52}]

In [86]:
def sim_sort(x):
    return x['similarity']

In [89]:
gg.sort(key=sim_sort, reverse=True)

In [90]:
gg

[{'name': 'a', 'similarity': 0.99},
 {'name': 'a', 'similarity': 0.52},
 {'name': 'a', 'similarity': 0.24},
 {'name': 'a', 'similarity': 0.2}]

In [79]:
cosine_similarity(features[3], data.to_dict('records')[3]['features'])

array([[0.99999994]], dtype=float32)

In [46]:
c = pd.read_pickle('d.pkl')

In [51]:
z = pd.read_csv('d.csv')

In [52]:
z.head()

,FileName,features
0,../../api-test-images/7688415.411.jpg,"[array([-1.9931037 , -1.8752654 , -2.9238546 ,..."
1,../../api-test-images/7688415.93554.jpg,"[array([-1.5990303, -2.5206456, -1.9668546, ....."
2,../../api-test-images/8016025.7294.jpg,"[array([ 0.24607146, -2.8339028 , -1.4787565 ,..."
3,../../api-test-images/8021256.585.jpg,"[array([-1.8691487 , -0.11222146, -1.5841146 ,..."
4,../../api-test-images/8021257.158.jpg,"[array([-0.8288594 , -0.22768264, -1.1380577 ,..."


In [53]:
c.head()

,FileName,features
0,../../api-test-images/7688415.411.jpg,"[[-1.9931037, -1.8752654, -2.9238546, 0.274329..."
1,../../api-test-images/7688415.93554.jpg,"[[-1.5990303, -2.5206456, -1.9668546, 0.257537..."
2,../../api-test-images/8016025.7294.jpg,"[[0.24607146, -2.8339028, -1.4787565, 0.940413..."
3,../../api-test-images/8021256.585.jpg,"[[-1.8691487, -0.112221465, -1.5841146, 0.0016..."
4,../../api-test-images/8021257.158.jpg,"[[-0.8288594, -0.22768264, -1.1380577, -1.1019..."


In [54]:
data.head()

,FileName,features
0,../../api-test-images/7688415.411.jpg,"[[-1.9931037, -1.8752654, -2.9238546, 0.274329..."
1,../../api-test-images/7688415.93554.jpg,"[[-1.5990303, -2.5206456, -1.9668546, 0.257537..."
2,../../api-test-images/8016025.7294.jpg,"[[0.24607146, -2.8339028, -1.4787565, 0.940413..."
3,../../api-test-images/8021256.585.jpg,"[[-1.8691487, -0.112221465, -1.5841146, 0.0016..."
4,../../api-test-images/8021257.158.jpg,"[[-0.8288594, -0.22768264, -1.1380577, -1.1019..."


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

pil_im = Image.open(filename[3]) #Take jpg + png
## Uncomment to open from URL
#import requests
#r = requests.get('https://www.vegvesen.no/public/webkamera/kamera?id=131206')
#pil_im = Image.open(BytesIO(r.content))
im_array = np.asarray(pil_im)
plt.imshow(im_array)
plt.show()

In [ ]:
pil_im = Image.open(filename[4]) #Take jpg + png
## Uncomment to open from URL
#import requests
#r = requests.get('https://www.vegvesen.no/public/webkamera/kamera?id=131206')
#pil_im = Image.open(BytesIO(r.content))
im_array = np.asarray(pil_im)
plt.imshow(im_array)
plt.show()

# Understand Vgg

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        VGG = models.vgg19(pretrained=True)
        self.feature = VGG.features
        self.classifier = nn.Sequential(*list(VGG.classifier.children())[:-3])
        pretrained_dict = VGG.state_dict()
        model_dict = self.classifier.state_dict()
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        model_dict.update(pretrained_dict)
        self.classifier.load_state_dict(model_dict)
 
    def forward(self, x):
        output = self.feature(x)
        output = output.view(output.size(0), -1)
        output = self.classifier(output)
        return output

In [ ]:
VGG = models.vgg19(pretrained=True)

In [ ]:
feature = VGG.features

In [ ]:
feature

In [ ]:
list(VGG.classifier.children())

In [ ]:
classifier = nn.Sequential(*list(VGG.classifier.children()))

In [ ]:
classifier

In [ ]:
pretrained_dict = VGG.state_dict()

In [ ]:
model_dict = classifier.state_dict()

In [ ]:
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}

In [ ]:
pretrained_dict

In [ ]:
model_dict.update(pretrained_dict)

In [ ]:
classifier.load_state_dict(model_dict)

In [55]:
BOOTS_DATA_PATH = '../../data/boots/'

In [57]:
BOOTS_DATA_PATH.split("/")[-2]

'boots'

In [58]:
DB_PATH = '../db/'

In [60]:
os.path.join(DB_PATH, BOOTS_DATA_PATH.split("/")[-2] + ".pkl")

'../db/boots.pkl'

In [65]:
x = os.walk(BOOTS_DATA_PATH)

In [62]:
x

<generator object walk at 0x0000015E73B59580>

In [66]:
for path, d, filelist in x:
    for filename in filelist:
        file_glob = os.path.join(path, filename)
        print(file_glob)
    print("---"*20)

../../data/boots/7688415.411.jpg
../../data/boots/7688415.93554.jpg
../../data/boots/7688434.324313.jpg
../../data/boots/7688434.355364.jpg
../../data/boots/7688434.60041.jpg
../../data/boots/7688520.253817.jpg
../../data/boots/7688520.3.jpg
../../data/boots/7688520.4854.jpg
../../data/boots/7688520.84362.jpg
../../data/boots/7688520.93554.jpg
../../data/boots/7811851.3.jpg
../../data/boots/7811851.6.jpg
../../data/boots/7877187.904.jpg
../../data/boots/7920102.216.jpg
../../data/boots/7920102.236038.jpg
../../data/boots/7920102.72.jpg
../../data/boots/7983495.341.jpg
../../data/boots/7983530.3.jpg
../../data/boots/7983530.5578.jpg
../../data/boots/7983530.66261.jpg
../../data/boots/7983530.76119.jpg
../../data/boots/7983551.341.jpg
../../data/boots/7983551.66261.jpg
../../data/boots/7987475.158.jpg
../../data/boots/7987475.18.jpg
../../data/boots/7987475.396.jpg
../../data/boots/7987475.607.jpg
../../data/boots/7987480.1137.jpg
../../data/boots/7987480.80.jpg
../../data/boots/7987480.